## Обкачивание страницы с помощью selenium

#### Подключаемся к странице и открываем оригинальный текст:

In [16]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager


In [17]:
import re

In [356]:
#driver = webdriver.Chrome('C:/Users/Алина/Desktop/прога/курсач/.wdm/drivers/chromedriver/win32/97.0.4692.71/chromedriver.exe')
driver = webdriver.Chrome('C:/Users/Алина/Desktop/прога/курсач/chromedriver.exe')

C:\Users\Алина\AppData\Local\Temp/ipykernel_14468/1276585012.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:/Users/Алина/Desktop/прога/курсач/chromedriver.exe')


In [535]:
driver.quit()

In [527]:
# не забыть сюда добавить тайм-ауты при скачивании большого количества песен
#driver.get("https://lyricstranslate.com/ru/next-level-sleduyushchiy-uroven.html#songtranslation")
driver.get("https://lyricstranslate.com/ru/catallena-%EA%B9%8C%ED%83%88%EB%A0%88%EB%82%98-%D0%BA%D0%B0%D1%82%D0%B0%D0%BB%D0%BB%D0%B5%D0%BD%D0%B0.html")

In [528]:
org_lyr_button = driver.find_element_by_id('lyrics-preview')

C:\Users\Алина\AppData\Local\Temp/ipykernel_14468/2693645806.py:1: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  org_lyr_button = driver.find_element_by_id('lyrics-preview')


In [529]:
org_lyr_button.click()

#### Скачиваем текст:

In [494]:
from collections import defaultdict
from pprint import pprint

In [495]:
def downloading_lyrics(drvr):
    error = False
    i = 0 # параграф
    j = 1 # строка
    endcount = 0
    lyrics = defaultdict(list)

    while True:
        cur = 'll-' + str(i) + '-' + str(j)
        line = drvr.find_elements_by_class_name(cur)
        if len(line) == 0:
            if endcount == 0: # закончился абзац 
                i += 1
                j = 1
                endcount = 1
            elif endcount == 1: # закончилась песня
                endcount = 2
            elif endcount == 2:
                break
        elif len(line) == 1:
            error = True
            j += 1
            endcount = 0
        else:
            lyrics[cur.strip('ll-')] = [line[0].text, line[1].text]
            j += 1
            endcount = 0
            
    return lyrics, error

In [530]:
lyrics = downloading_lyrics(driver)

C:\Users\Алина\AppData\Local\Temp/ipykernel_14468/3132871542.py:10: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  line = drvr.find_elements_by_class_name(cur)


#### Скачиваем мета-инфу:

(при добавлении в файл надо будет проверять, что не вернулось None и заменять на not found если не нашлось)

In [531]:
meta = {}

translation_node = driver.find_element_by_class_name('translate-node-text').text
song_node = driver.find_element_by_class_name('song-node-text').text

pattern_org = 'Добавлено .*? в ..\, [0-9][0-9]\/[0-9][0-9]\/[0-9][0-9][0-9][0-9] \- [0-9][0-9]:[0-9][0-9]'
pattern_tr = 'Добавлено .*? в ..\, [0-9][0-9]\/[0-9][0-9]\/[0-9][0-9][0-9][0-9] \- [0-9][0-9]:[0-9][0-9]'

org_meta = re.search(pattern_org, song_node)
tr_meta = re.search(pattern_tr, translation_node)

org_date = re.search('[0-9][0-9]\/[0-9][0-9]\/[0-9][0-9][0-9][0-9]', org_meta[0])[0]
tr_date = re.search('[0-9][0-9]\/[0-9][0-9]\/[0-9][0-9][0-9][0-9]', tr_meta[0])[0]
translator = re.search('Добавлено .*? в', tr_meta[0])[0][10:-2]

meta['original_song_date'] = org_date
meta['translation_date'] = tr_date
meta['translator'] = translator

# вот это бы надо доделать, там ещё куча ерунды бывает, надо брать либо до • либо до \n
meta['artist'] = driver.find_element_by_class_name(
    'song-node-info-artist').text.split(
    '•')[0].strip('Исполнитель: ')

meta['song'] = driver.find_element_by_class_name(
    'song-node-info-album').text.split(
    '•')[0].strip('Песня: ')

meta['album'] = driver.find_element_by_class_name(
    'song-node-info-album').text.split(
    '•')[1].strip('Альбом: ')

C:\Users\Алина\AppData\Local\Temp/ipykernel_14468/3497952685.py:3: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  translation_node = driver.find_element_by_class_name('translate-node-text').text
C:\Users\Алина\AppData\Local\Temp/ipykernel_14468/3497952685.py:4: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  song_node = driver.find_element_by_class_name('song-node-text').text
C:\Users\Алина\AppData\Local\Temp/ipykernel_14468/3497952685.py:21: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  meta['artist'] = driver.find_element_by_class_name(
C:\Users\Алина\AppData\Local\Temp/ipykernel_14468/3497952685.py:25: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  meta['song'] = driver.find_element_by_class_name(
C:\Users\Алина\AppData\Local\Temp/ipykernel_14468/3497952685.py:29: DeprecationWar

In [532]:
pprint(meta)

{'album': 'THE THIRD SINGLE CATALLENA',
 'artist': 'Orange Caramel (오렌지캬라멜)',
 'original_song_date': '17/03/2017',
 'song': '까탈레나 (Catallena)',
 'translation_date': '28/11/2018',
 'translator': '8U'}


In [533]:
pprint(lyrics[0])

defaultdict(<class 'list'>,
            {'0-1': ['Маленькая танцующая Каталлена',
                     '춤추는 작은 까탈레나 (Red Sun)\n1'],
             '0-2': ['Я влюбляюсь в тебя, сама того не замечая',
                     '나도 모르게 빠져 들어가'],
             '0-3': ['Красивая и гордая Кателлена (красное солнце)',
                     '도도한 콧대 까탈레나 (Red Sun)'],
             '0-4': ['Ютти-мери-хой-хой*, я околдована',
                     'Jutti Meri Oye Hoi Hoi 홀려 들어가'],
             '0-5': ['Ох боже, она такая шикарная, я влюбилась',
                     '어머머머 멋있어 반했어'],
             '0-6': ['Даже будучи девушкой, я вижу, как она прекрасна',
                     '어쩜 같은 여자끼리 봐도 참 멋있어'],
             '0-7': ['Она своенравна, но я хочу снова увидеть ее',
                     '까칠까칠 하지만 한 번만'],
             '0-8': ['Хочу узнать ее, хочу танцевать с ней',
                     '보고파 알고파 함께 춤 추고파'],
             '1-1': ['Ла-ла-ла, махни, махни своей рукой', '랄라랄라 흔들흔들 손 흔들고'],
             '1-2': ['Кричи,

#### Загрузка текстов в файл

konlpy не пожелал устанавливаться на мой ноутбук (проблемы с Microsoft Visual C++, которые мы не смогли решить в своё время на программировании), а в коллабе не очень понятно, как подключить драйвер к браузеру, поэтому в этой программе я ещё и скачаю всё это в файл .json, который буду считывать уже в коллабе для морфологической обработки и создания XML.

Тут нужно ещё будет отдельно подумать, как передавать мета-информацию.

In [509]:
import json

In [534]:
file_name = meta['song'] + ' by ' + meta['artist']
with open(file_name + ".json", "w", encoding='utf-8') as write_file:
    json.dump([meta, lyrics[0]], write_file, ensure_ascii=False, indent=3)

Дальнейшая работа происходит здесь: https://colab.research.google.com/drive/1pcikDOeSAy7rGK81xCR8o9TGQOFl1SPP#scrollTo=b-ORbudQTsiy